## **Dependencies**

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

## **Load JSON**

Choose respective json file to transform to data frame
Json be found in /src/response_[ORG]

### Define Params

In [3]:
##########
# PARAMS #
##########

abbreviation = "wb" # abbreviation of development bank
src_folder = "../../src/"
response_file = src_folder + f"response_{abbreviation}.json" # json format -> fetched in iati.ipynb
output_file = src_folder + f"transformed_{abbreviation}.csv"

In [4]:
#Read in data into df from json
df = pd.read_json(response_file)
df.head(10)

,sector_code,iati_identifier,title_narrative,document_link_url,reporting_org_ref,sector_vocabulary,activity_date_type,activity_status_code,description_narrative,last_updated_datetime,recipient_region_code,activity_date_iso_date,location_name_narrative,reporting_org_narrative,recipient_country_code
0,"[23111, LP, 000212, 000141, 000021, 000014]",44000-P101414,[Eastern Caribbean Energy Regulatory Authority...,[http://documents.worldbank.org/curated/en/479...,44000,"[1, 99, 98, 98, 98, 98]","[1, 2, 3, 4]",3,[The objective of the Project is to establish ...,2024-01-27T18:20:31Z,[389],"[2007-08-03T00:00:00Z, 2011-06-16T00:00:00Z, 2...",[Castries],[World Bank],NaN
1,"[15126, 22040, CF, CS, CZ, BC, 000014, 000114,...",44000-P117087,[OECS E-Government for Regional Integration - ...,[http://documents.worldbank.org/curated/en/636...,44000,"[1, 1, 99, 99, 99, 99, 98, 98, 98, 98, 98, 98,...","[1, 2, 3, 4]",3,[The overall development objective of the proj...,2024-01-27T18:20:33Z,[389],"[2009-07-02T00:00:00Z, 2009-12-15T00:00:00Z, 2...",[Parish of Saint George],[World Bank],NaN
2,"[74020, WZ, BZ, TZ, TV, WF, 000751, 000626, 00...",44000-P117871,[Regional Disaster Vulnerability Reduction APL...,[http://documents.worldbank.org/curated/en/755...,44000,"[1, 99, 99, 99, 99, 99, 98, 98, 98, 98, 98, 98...","[1, 2, 3, 4]",3,[The Program aims at measurably reducing vulne...,2024-01-27T18:20:34Z,[389],"[2011-03-10T00:00:00Z, 2011-06-23T00:00:00Z, 2...","[Gouyave, Kingstown, Union Island, Arnos Vale,...",[World Bank],NaN
3,"[33210, 41040, 22040, 21040, CI, YF, YT, TP, T...",44000-P152117,[OECS Regional Tourism Competitiveness],[http://documents.worldbank.org/curated/en/172...,44000,"[1, 1, 1, 1, 99, 99, 99, 99, 99, 98, 98, 98, 9...","[1, 2, 3, 4]",2,[The project development objective is to incre...,2024-01-27T18:20:38Z,[389],"[2015-04-30T00:00:00Z, 2017-04-06T00:00:00Z, 2...","[Grenada, Fort George, Carriacou Island, The C...",[World Bank],NaN
4,"[32130, 24030, FK, FP, 000081, 000022, 000026,...",44000-P157715,[OECS MSME Guarantee Facility Project],[http://documents.worldbank.org/curated/en/099...,44000,"[1, 1, 99, 99, 98, 98, 98, 98, 98, 98, 98, 98,...","[1, 2, 3, 4]",2,[The development objective is to facilitate ad...,2024-01-27T18:20:40Z,[389],"[2016-12-05T00:00:00Z, 2018-07-05T00:00:00Z, 2...","[Antigua and Barbuda, Grenada, Dominica, Feder...",[World Bank],NaN
5,"[31191, 31194, YA, YF, AB, 000072, 000754, 000...",44000-P158958,[OECS Regional Agriculture Competitiveness Pro...,[http://documents.worldbank.org/curated/en/278...,44000,"[1, 1, 99, 99, 99, 98, 98, 98, 98, 98, 98, 98,...","[1, 2, 3, 4]",3,[The proposed Project Development Objective (P...,2024-01-27T18:20:41Z,[389],"[2017-03-15T00:00:00Z, 2017-05-25T00:00:00Z, 2...","[Saint Patrick, Saint Mark, Saint John, Saint ...",[World Bank],NaN
6,"[21023, 74020, 14021, WZ, WC, TZ, BZ, 000753, ...",44000-P129992,[Third Phase Disaster Vuln.Reduction APL for D...,[http://documents.worldbank.org/curated/en/441...,44000,"[1, 1, 1, 99, 99, 99, 99, 98, 98, 98, 98, 98, 98]","[1, 2, 3, 4]",2,[The objective of the Project is to reduce vul...,2024-01-27T22:53:40Z,NaN,"[2013-05-13T00:00:00Z, 2014-05-01T00:00:00Z, 2...","[Saint Mark, Saint John, Saint George, Saint D...",[World Bank],[DM]
7,"[23260, 23630, LI, 000086, 000863, 000081, 000...",44000-P162149,[Dominica Geothermal Risk Mitigation Project],[http://documents.worldbank.org/curated/en/870...,44000,"[1, 1, 99, 98, 98, 98, 98]","[1, 2, 3, 4]",2,[The objective of the proposed Project is to h...,2024-01-27T22:53:41Z,NaN,"[2019-03-18T00:00:00Z, 2019-03-18T00:00:00Z, 2...","[Trafalgar, Roseau, Wotten Waven, Laudat]",[World Bank],[DM]
8,"[31166, 31120, 31110, 31150, 31320, 31163, 311...",44000-P166328,[Emergency Agricultural Livelihoods and Climat...,[http://documents.worldbank.org/curated/en/676...,44000,"[1, 1, 1, 1, 1, 1, 1, 99, 99, 99, 99, 99, 99, ...","[1, 2, 3, 4]",2,[The Project Development Objective (PDO) is to...,2024-01-27T22:53:41Z,NaN,"[2018-03-29T

In [5]:
print(f"Entries: {len(df)}")
df.reporting_org_ref.value_counts()

Entries: 4296


44000    4296
Name: reporting_org_ref, dtype: int64

In [6]:
# create new empty df to fill with transformed data

trans_df = pd.DataFrame()

## **Feature Transformation & Engineering**

### IATI ID

In [7]:
trans_df["iati_id"] = df["iati_identifier"].values

trans_df.head(2)

,iati_id
0,44000-P101414
1,44000-P117087


### Title

In [8]:
################
# Add en title #
################

pbar = tqdm(total=len(df))
trans_df["title_en"] = "NaN"

for index, row in df.iterrows():
    title_row = row['title_narrative']
    try:
        if 'title_narrative_xml_lang' in df.columns:
            lang_list = row['title_narrative_xml_lang']

            # nan in pandas is type float
            # check if nan and if yes take first entry in lang
            if isinstance(lang_list, float):
                if isinstance(title_row, float):
                    trans_df["title_en"][index] = "NaN"
                else:
                    trans_df["title_en"][index] = title_row[0]
            elif len(lang_list) == len(title_row):
                for j in range(0, len(lang_list)):
                    if "en" or "EN" in lang_list:
                        if lang_list[j].lower() == "en":
                            title = title_row[j]
                            trans_df["title_en"][index] = title
                    else:
                        trans_df["title_en"][index] = "NaN"
            else:
                pass
        else:
            trans_df["title_en"][index] = row['title_narrative'][0]
    except:
        print(f"Error: Index: {index}, Row: {lang_list}, {title_row}")
    
    pbar.update(1)

pbar.close()

trans_df.head(2)

  0%|          | 0/4296 [00:00<?, ?it/s]

,iati_id,title_en
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...
1,44000-P117087,OECS E-Government for Regional Integration - S...


In [9]:
###################
# Add other title #
###################

pbar = tqdm(total=len(df))

trans_df["title_other"] = "NaN"

for index, row in df.iterrows():
    try:
        if 'title_narrative_xml_lang' in df.columns:
            lang_list = row['title_narrative_xml_lang']

            # every title which is has no lang attribute is classified as English and therefore not in other
            if isinstance(lang_list, float):
                trans_df["title_other"][index] = "NaN"
            elif len(lang_list) == len(title_row):
                for j in range(0, len(lang_list)):
                    if lang_list[j].lower() != "en":
                        title = row['title_narrative'][j]
                        if trans_df["title_other"][index] == "NaN":
                            trans_df["title_other"][index] = title
                        else:
                            trans_df["title_other"][index] = f"{trans_df['title_other'][index]}; {title}"
            else:
                trans_df["title_other"][index] = title
            
            pbar.update(1)

        else:
            trans_df["title_other"][index] = "NaN"
        
    except Exception as e:
        print(f"Error: Index: {index} \n Row: {row} \n Exception: {e}")

pbar.close()

trans_df.head(200)

  0%|          | 0/4296 [00:00<?, ?it/s]

,iati_id,title_en,title_other
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN
2,44000-P117871,Regional Disaster Vulnerability Reduction APL1...,NaN
3,44000-P152117,OECS Regional Tourism Competitiveness,NaN
4,44000-P157715,OECS MSME Guarantee Facility Project,NaN
...,...,...,...
195,44000-P175325,Casamance Economic Development Project,NaN
196,44000-P175830,Stormwater Management and Climate Change Adapt...,NaN
197,44000-P175915,Senegal: Natural Resources Management Project,NaN
198,44000-P176419,Enhancing Connectivity in the Northern and Cen...,NaN


### Organization

In [10]:
trans_df['organization'] = df['reporting_org_narrative'].apply(lambda x: x[0])

print(trans_df.organization.value_counts())
trans_df.head(2)

World Bank    4296
Name: organization, dtype: int64


,iati_id,title_en,title_other,organization
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN,World Bank
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN,World Bank


### Country

In [11]:
trans_df["country"] = df["recipient_country_code"]
trans_df.head(5)

,iati_id,title_en,title_other,organization,country
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN,World Bank,NaN
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN,World Bank,NaN
2,44000-P117871,Regional Disaster Vulnerability Reduction APL1...,NaN,World Bank,NaN
3,44000-P152117,OECS Regional Tourism Competitiveness,NaN,World Bank,NaN
4,44000-P157715,OECS MSME Guarantee Facility Project,NaN,World Bank,NaN


### Region

In [12]:
trans_df['region'] = df['recipient_region_code']
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN,World Bank,NaN,[389]
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN,World Bank,NaN,[389]
2,44000-P117871,Regional Disaster Vulnerability Reduction APL1...,NaN,World Bank,NaN,[389]
3,44000-P152117,OECS Regional Tourism Competitiveness,NaN,World Bank,NaN,[389]
4,44000-P157715,OECS MSME Guarantee Facility Project,NaN,World Bank,NaN,[389]


### Location

In [13]:
if 'title_narrative_xml_lang' in df.columns:
    trans_df['location'] = df['location_name_narrative']
else:
    trans_df['location'] = "NaN"
    
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region,location
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN,World Bank,NaN,[389],NaN
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN,World Bank,NaN,[389],NaN
2,44000-P117871,Regional Disaster Vulnerability Reduction APL1...,NaN,World Bank,NaN,[389],NaN
3,44000-P152117,OECS Regional Tourism Competitiveness,NaN,World Bank,NaN,[389],NaN
4,44000-P157715,OECS MSME Guarantee Facility Project,NaN,World Bank,NaN,[389],NaN


### Descriptions

descr1, descri2, ... can have overlap. Doppelte Beschriebungen entfernen! Es gibt lib um overlap heruaszufinden!

In [14]:
#################################
# Description english and other #
#################################

trans_df["description_en"] = "NaN"
trans_df["description_other"] = "NaN"

for index, row in df.iterrows():

    try:
        if 'description_narrative_xml_lang' in df.columns:
            descr_list = row['description_narrative_xml_lang']
            descr_row = row['description_narrative']

            # nan in pandas is type float
            # check if nan and if yes take first entry in descr
            if isinstance(descr_list, float):
                if isinstance(descr_row, float):
                    trans_df["description_en"][index] = "NaN"
                else:
                    trans_df["description_en"][index] = descr_row[0]
            else:
                # iterate throug description list
                for j in range(0, len(descr_list)):
                    # if description english
                    if descr_list[j].lower() == "en":
                        if type(descr_row) == float:
                            descr = "NaN"
                        else:
                            descr = descr_row[j]
                        if trans_df["description_en"][index] == "NaN":
                            trans_df["description_en"][index] = descr
                        else:
                            trans_df["description_en"][index] = f"{trans_df['description_en'][index]}; {descr}"
                    else:
                        if type(descr_row) == float:
                            descr = "NaN"
                        else:
                            descr = descr_row[j]
                        if trans_df["description_other"][index] == "NaN":
                            trans_df["description_other"][index] = descr
                        else:
                            trans_df["description_other"][index] = f"{trans_df['description_other'][index]}; {descr}"
        else:
            descr_str = ""
            for d in row['description_narrative']:
                descr_str += f"{d}; "
            trans_df["description_en"][index] = descr_str

    except:
        print(f"Error: Index: {index} \n Row: {row}")
        
trans_df.head(5)

Error: Index: 264 
 Row: sector_code                                                     [FP, 000031]
iati_identifier                                                44000-P180861
title_narrative            [Financial Sector Safety Net Strengthening Pro...
document_link_url          [http://documents.worldbank.org/curated/en/099...
reporting_org_ref                                                      44000
sector_vocabulary                                                   [99, 98]
activity_date_type                                              [1, 2, 3, 4]
activity_status_code                                                       2
description_narrative                                                    NaN
last_updated_datetime                                   2024-01-27T23:42:51Z
recipient_region_code                                                    NaN
activity_date_iso_date     [2023-11-09T00:00:00Z, 2023-11-09T00:00:00Z, 2...
location_name_narrative                            

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN,World Bank,NaN,[389],NaN,The objective of the Project is to establish a...,NaN
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN,World Bank,NaN,[389],NaN,The overall development objective of the proje...,NaN
2,44000-P117871,Regional Disaster Vulnerability Reduction APL1...,NaN,World Bank,NaN,[389],NaN,The Program aims at measurably reducing vulner...,NaN
3,44000-P152117,OECS Regional Tourism Competitiveness,NaN,World Bank,NaN,[389],NaN,The project development objective is to increa...,NaN
4,44000-P157715,OECS MSME Guarantee Facility Project,NaN,World Bank,NaN,[389],NaN,The development objective is to facilitate add...,NaN


### Status

In [15]:
# https://iatistandard.org/en/iati-standard/203/codelists/activitystatus/
activity_status = {
    1: "Pipeline/identification",
    2: "Implementation",
    3: "Finalisation",
    4: "Closed",
    5: "Cancelled",
    6: "Suspended"
}

trans_df["status"] = df.activity_status_code
trans_df['status'] = trans_df['status'].replace(activity_status)

trans_df.head(2)
    

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN,World Bank,NaN,[389],NaN,The objective of the Project is to establish a...,NaN,Finalisation
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN,World Bank,NaN,[389],NaN,The overall development objective of the proje...,NaN,Finalisation


### Date

In [16]:
#############
# Date Type #
#############

# One Hot
# 1 -> Yes
# 0 -> No

# Codes:
# 1 Planned start
# 2 Actual start
# 3 Planned end
# 4 Actual end

trans_df["planned_start"] = "NaN"
trans_df["actual_start"] = "NaN"
trans_df["planned_end"] = "NaN"
trans_df["actual_end"] = "NaN"

date_types = {
    1: "planned_start",
    2: "actual_start",
    3: "planned_end",
    4: "actual_end"
}

for index, row in df.iterrows():
    dtype_list = row["activity_date_type"]
    iso_date_list = row["activity_date_iso_date"]

    combined_list = list(zip(dtype_list, iso_date_list))

    # replace nums with column names from date_types
    combined_list = [(date_types[int(t[0])], t[1]) for t in combined_list]

    for i in combined_list:
        trans_df[i[0]] = i[1]

trans_df.head(1800)


,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN,World Bank,NaN,[389],NaN,The objective of the Project is to establish a...,NaN,Finalisation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN,World Bank,NaN,[389],NaN,The overall development objective of the proje...,NaN,Finalisation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z
2,44000-P117871,Regional Disaster Vulnerability Reduction APL1...,NaN,World Bank,NaN,[389],NaN,The Program aims at measurably reducing vulner...,NaN,Finalisation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z
3,44000-P152117,OECS Regional Tourism Competitiveness,NaN,World Bank,NaN,[389],NaN,The project development objective is to increa...,NaN,Implementation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z
4,44000-P157715,OECS MSME Guarantee Facility Project,NaN,World Bank,NaN,[389],NaN,The development objective is to facilitate add...,NaN,Implementation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,44000-P170384,ETHIOPIA CLIMATE ACTION THROUGH LANDSCAPE MANA...,NaN,World Bank,[ET],NaN,NaN,The Program Development Objective (PDO) is to ...,NaN,Implementation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z
1796,44000-P171034,Ethiopia Digital Foundations Project,NaN,World Bank,[ET],NaN,NaN,The Project Development Objective is to increa...,NaN,Implementation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z
1797,44000-P171742,Access to Distributed Electricity and Lighting...,NaN,World Bank,[ET],NaN,NaN,The development objective is to increase acces...,NaN,Implementation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z
1798,44000-P172284,Ethiopia Human Capital Operation,NaN,World Bank,[ET],NaN,NaN,The objective of the Operation is to improve l...,NaN,Implementation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z


### Last Update

In [17]:
trans_df['last_update'] = df['last_updated_datetime']
trans_df.head(2)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN,World Bank,NaN,[389],NaN,The objective of the Project is to establish a...,NaN,Finalisation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z,2024-01-27T18:20:31Z
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN,World Bank,NaN,[389],NaN,The overall development objective of the proje...,NaN,Finalisation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z,2024-01-27T18:20:33Z


### Sector Codes (CRS & SGDs)

In [18]:
sector_codes = {
    1: "OECD DAC CRS Purpose Codes (5 digit)",
    2: "OECD DAC CRS Purpose Codes (3 digit)",
    3: "Classification of the Functions of Government (UN)",
    4: "Statistical classification of economic activities in the European Community",
    5: "National Taxonomy for Exempt Entities (USA)",
    6: "AidData",
    7: "SDG Goal",
    8: "SDG Target",
    9: "SDG Indicator",
    10: "Humanitarian Global Clusters (Inter-Agency Standing Committee)",
    11: "North American Industry Classification System (NAICS)",
    12: "UN System Function",
    99: "Reporting Organisation", # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity
    98: "Reporting Organisation 2" # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity (if they are referencing more than one)
}

In [19]:
crs5_df = pd.read_csv("../../src/crs5_codes.csv")
crs5_df.head(2)

,code,name,description,language,category,category-name,category-description
0,11110,Education policy and administrative management,"Education sector policy, planning and programm...",en,111,"Education, Level Unspecified",The codes in this category are to be used only...
1,11120,Education facilities and training,"Educational buildings, equipment, materials; s...",en,111,"Education, Level Unspecified",The codes in this category are to be used only...


In [20]:
crs3_df = pd.read_csv("../../src/crs3_codes.csv")
crs3_df.head(2)

,code,name,description,language,category,category-name,category-description
0,111,"Education, Level Unspecified",The codes in this category are to be used only...,en,NaN,NaN,NaN
1,112,Basic Education,NaN,en,NaN,NaN,NaN


#### Get CRS5 and CRS3 Tags

other tags are not given by organizations in IATI data; need other ways to find out these tags

In [21]:
# to process different variants of codes (crs3, crs5 etc.)

def process_codes(combined_list, translation_df, code_index):
    code_text = ""
    codes_nums = ""
    if any(item[0] == code_index for item in combined_list):
        for i in combined_list:
            if i[0] == code_index:
                translation = translation_df.loc[translation_df['code'] == int(i[1]), 'name'].values[0]
                code_text += f"{translation}; "
                codes_nums += f"{i[1]}; "
        return code_text, codes_nums
    else:
        return "NaN", "NaN"
    
def derive_crs3(combined_list, translation_df, code_index="1"):
    code_text = ""
    codes_nums = ""
    if any(item[0] == code_index for item in combined_list):
        for i in combined_list:
            if i[0] == code_index:
                print(i[1][:3])
                translation = translation_df.loc[translation_df['code'] == int(i[1][:3]), 'name'].values[0]
                code_text += f"{translation}; "
                codes_nums += f"{i[1][:3]}; "
        return code_text, codes_nums
    else:
        return "NaN", "NaN"

In [22]:
###############
# ECTRACT CRS #
###############
trans_df["crs_5_code"] = "NaN"
trans_df["crs_5_name"] = "NaN"

# Most Project dont have information on crs3 -> crs3 derived from crs5 tags
trans_df["crs_3_code"] = "NaN"
trans_df["crs_3_name"] = "NaN"


for index, row in df.iterrows():
    crs_voc_list = row['sector_vocabulary']
    crs_code_list = row['sector_code']

    if type(crs_voc_list) == float:
        pass
    else:
        try:
            combined_list = list(zip(crs_voc_list, crs_code_list))

            # CRS 5
            crs5_str, crs5_codes = process_codes(combined_list, crs5_df, "1")
            trans_df["crs_5_code"][index] = crs5_codes
            trans_df["crs_5_name"][index] = crs5_str
            
            # CRS 3
            crs3_str, crs3_codes = derive_crs3(combined_list, crs3_df)
            trans_df["crs_3_code"][index] = crs3_codes
            trans_df["crs_3_name"][index] = crs3_str

        except:
            print(f"Error on Index {index}")
            pass

trans_df.head(5)

231
151
220
740
332
410
220
210
321
240
311
311
210
740
140
232
236
311
311
311
311
313
311
311
740
160
730
210
151
121
740
160
332
220
250
151
220
220
740
220
122
231
121
130
121
151
151
331
321
140
140
151
160
210
140
140
233
232
240
311
160
210
740
121
122
122
130
160
210
151
220
231
236
322
322
122
112
160
111
111
111
130
140
140
121
122
121
151
130
130
140
140
140
311
311
311
311
311
730
740
113
151
151
740
311
311
210
311
730
236
231
151
111
231
121
114
232
121
121
122
122
112
112
111
111
111
160
122
113
332
250
331
321
140
140
210
210
122
140
140
140
140
151
151
151
250
122
160
312
312
740
151
410
430
151
111
114
114
311
311
311
311
311
311
111
111
111
112
151
250
323
210
121
121
122
430
730
740
140
140
140
236
321
151
151
151
210
210
140
410
430
430
430
140
140
311
210
140
740
111
236
240
114
210
140
740
140
430
140
311
140
740
311
430
430
410
220
210
410
312
230
740
410
311
210
140
140
740
430
210
311
111
114
312
430
313
311
311
210
430
160
430
140
430
160
321
311
220
410
740


,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN,World Bank,NaN,[389],NaN,The objective of the Project is to establish a...,NaN,Finalisation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z,2024-01-27T18:20:31Z,23111;,"Energy sector policy, planning and administrat...",231;,Energy Policy;
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN,World Bank,NaN,[389],NaN,The overall development objective of the proje...,NaN,Finalisation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z,2024-01-27T18:20:33Z,15126; 22040;,Other general public services; Information and...,151; 220;,Government & Civil Society-general; Communicat...
2,44000-P117871,Regional Disaster Vulnerability Reduction APL1...,NaN,World Bank,NaN,[389],NaN,The Program aims at measurably reducing vulner...,NaN,Finalisation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z,2024-01-27T18:20:34Z,74020;,Multi-hazard response preparedness;,740;,Disaster Prevention & Preparedness;
3,44000-P152117,OECS Regional Tourism Competitiveness,NaN,World Bank,NaN,[389],NaN,The project development objective is to increa...,NaN,Implementation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z,2024-01-27T18:20:38Z,33210; 41040; 22040; 21040;,Tourism policy and administrative management; ...,332; 410; 220; 210;,Tourism; General Environment Protection; Commu...
4,44000-P157715,OECS MSME Guarantee Facility Project,NaN,World Bank,NaN,[389],NaN,The development objective is to facilitate add...,NaN,Implementation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z,2024-01-27T18:20:40Z,32130; 24030;,Small and medium-sized enterprises (SME) devel...,321; 240;,Industry; Banking & Financial Services;


#### Derive crs3 tags from crs5 tags

It is possible to derive CRS3 tags from CRS5 tags by looking at the first 3 characters.

### Documents

In [23]:
df.document_link_url.value_counts()

trans_df['docs'] = df['document_link_url']
trans_df.head(2)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs
0,44000-P101414,Eastern Caribbean Energy Regulatory Authority ...,NaN,World Bank,NaN,[389],NaN,The objective of the Project is to establish a...,NaN,Finalisation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z,2024-01-27T18:20:31Z,23111;,"Energy sector policy, planning and administrat...",231;,Energy Policy;,[http://documents.worldbank.org/curated/en/479...
1,44000-P117087,OECS E-Government for Regional Integration - S...,NaN,World Bank,NaN,[389],NaN,The overall development objective of the proje...,NaN,Finalisation,2009-09-29T00:00:00Z,2010-04-29T00:00:00Z,2021-05-31T00:00:00Z,2021-05-31T00:00:00Z,2024-01-27T18:20:33Z,15126; 22040;,Other general public services; Information and...,151; 220;,Government & Civil Society-general; Communicat...,[http://documents.worldbank.org/curated/en/636...


## **Save DF as CSV**

In [24]:
trans_df.to_csv(output_file, index=False) 